In [1]:
import os
import cv2
import numpy as np

In [2]:
filename = 'Si-test-0710-png-smo2'
origin_dir = os.path.join(filename, 'origin-451')
cut_dir = os.path.join(filename, 'cut-s128-451')

os.makedirs(origin_dir, exist_ok=True)
os.makedirs(cut_dir, exist_ok=True)

origin_files = os.listdir(origin_dir)

In [5]:
def cut_data(data, lx, ly, stride=(1, 1), standardize=False, cover_all = False):
    """
    cuts up the data into pieces with dimension lx-by-ly
    data = 2-dimensional array with integer elements ranging from 0 to num_classes-1
    """
    data_shape = np.shape(data)
    print(data_shape)
    (nx, ny) = (data_shape[0], data_shape[1])
    (sx, sy) = stride

    if lx > nx or ly > ny or sx > nx or sy > ny:
        print("Error: cut dimensions are bigger than the image")
        print(lx, ly)
        exit()

    x_list = np.arange(0, nx - lx + 1, sx)
    y_list = np.arange(0, ny - ly + 1, sy)
    if cover_all:
        if x_list[-1]+lx < nx:
            x_list = np.append(x_list, nx-lx)
        if y_list[-1]+ly < ny:
            y_list = np.append(y_list, ny-ly)
            
    cut_data = [ data[i:i+lx,j:j+ly] for j in y_list for i in x_list]

    if standardize:
        cut_data = [(x - np.mean(x))/np.std(x) for x in cut_data]

    return np.array(cut_data)

for origin in origin_files:
    pic_dir = os.path.join(origin_dir, origin)
    input_img = cv2.imread(pic_dir)
    input_img = np.array(input_img)
    # (size_x, size_y) = input_img.shape

    (lx, ly) = (256, 256)
    (sx, sy) = (128, 128)
    input_cuts = cut_data(input_img, lx, ly, (sx, sy), standardize=False, cover_all=True)
    count = 0
    for cut in input_cuts:
        save_dir = os.path.join(cut_dir, f'{origin[:-4]}_{count}.png')
        cv2.imwrite(save_dir, cut) 
        count += 1

(2048, 2048, 3)
